In [ ]:
!pip install yfinance statsmodels

In [ ]:
import yfinance as yf
import pandas as pd

# Download historical stock data
ticker = 'ADANIENSOL.NS'  
data = yf.download(ticker)

# Use the 'Close' price for modeling
prices = data['Close']

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Split the data into training and test sets
train_size = int(len(prices) * 0.8)
train, test = prices[:train_size], prices[train_size:]

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import itertools
import warnings

warnings.filterwarnings("ignore")

# Define the p, d, q parameters to take any value between 0 and 2
p = d = q = range(0, 3)

# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))

# Define a function to evaluate an ARIMA model
def evaluate_arima_model(train, test, arima_order):
    model = ARIMA(train, order=arima_order, trend='ct')
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=len(test))
    loss = mean_squared_error(test, predictions)
    return loss

# Tune hyperparameters
from sklearn.metrics import mean_squared_error

best_score, best_cfg = float("inf"), None
for param in pdq:
    try:
        loss = evaluate_arima_model(train, test, param)
        if loss < best_score:
            best_score, best_cfg = loss, param
        print(f'ARIMA{param} MSE={loss}')
    except:
        continue

print(f'Best ARIMA{best_cfg} MSE={best_score}')

ARIMA(0, 0, 0) MSE=21880.12605800165
ARIMA(0, 0, 1) MSE=21711.25367651068
ARIMA(0, 0, 2) MSE=21754.223254652672
ARIMA(1, 0, 0) MSE=16336.313183612185
ARIMA(1, 0, 1) MSE=18335.25614065999
ARIMA(1, 0, 2) MSE=17646.238963064963
ARIMA(2, 0, 0) MSE=18358.8218443162
ARIMA(2, 0, 1) MSE=17852.84346154247
ARIMA(2, 0, 2) MSE=17746.34936343994
Best ARIMA(1, 0, 0) MSE=16336.313183612185


In [ ]:
# Fit the best ARIMA model
model = ARIMA(prices, order=best_cfg)
model_fit = model.fit()

# Forecast for the next 1 day and 1 week
forecast_1d = model_fit.forecast(steps=1)
forecast_1w = model_fit.forecast(steps=7)

In [ ]:
print(f'1-Day Forecast: {forecast_1d.values}')
print(f'1-Week Forecast: {forecast_1w.values}')

1-Day Forecast: [1119.72947347]
1-Week Forecast: [1119.72947347 1114.73983923 1109.92479145 1105.27822137 1100.794234
 1096.46714059 1092.29145144]


In [ ]:
# Evaluate accuracy on the test set
test_predictions = model_fit.predict(start=train_size, end=len(prices)-1)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(test, test_predictions)
print(f'Test MSE: {mse}')

Test MSE: 1711.4472482835877
